# Visual Screener
**Jupyter notebook that scrapes stocks data from Screener.in and creates charts.**

In [43]:

# If Consolidated then value should be 1 else it should be 0 

consolidated = 0  # 0  or 1

stock_symbol = "ICICIBANK" 

if consolidated == 0: 
    stock_screener_url = "https://www.screener.in/company/"+stock_symbol+"/"
else:
    stock_screener_url = "https://www.screener.in/company/"+stock_symbol+"/consolidated/"


## 1. Part-1

In [44]:
import requests as rq
import pandas as pd 
import numpy as np
import time
import datetime
from bs4 import BeautifulSoup
import re
import os

from plotly.subplots import make_subplots
import plotly.offline as pyo 
import plotly.graph_objects as go

In [50]:
def csvfy(lines):
    import re

    # Line - 0
    if m := re.match(r"Market Cap ₹ ([0-9,.-]+) Cr.", lines[0]):
        line_0 = m.group(1).replace(',','')
    else: 
        line_0 = "NaN"

    # Line - 1
    if m := re.match(r"Current Price ₹ ([0-9,.-]+)", lines[1]):
        line_1 = m.group(1).replace(',','')
    else: 
        line_1 = "NaN"

    # Line - 2a
    if m := re.match(r"High \/ Low ₹ ([0-9,..-]+) \/ ([0-9,.]+)", lines[2]):
        line_2a = m.group(1).replace(',','')
    else: 
        line_2a = "NaN"

    # Line - 2b
    if m := re.match(r"High \/ Low ₹ ([0-9,.-]+) \/ ([0-9,.]+)", lines[2]):
        line_2b = m.group(2).replace(',','')
    else: 
        line_2b = "NaN"

    # Line - 3
    if m := re.match(r"Stock P\/E ([0-9,.-]+)", lines[3]):
        line_3 = m.group(1).replace(',','')
    else: 
        line_3 = "NaN"

    # Line - 4
    if m := re.match(r"Book Value ₹ ([0-9,.-]+)", lines[4]):
        line_4 = m.group(1).replace(',','')
    else: 
        line_4 = "NaN"

    # Line - 5
    if m := re.match(r"Dividend Yield ([0-9,.-]+) %", lines[5]):
        line_5 = m.group(1).replace(',','')
    else: 
        line_5 = "NaN"

    # Line - 6
    if m := re.match(r"ROCE ([0-9,.-]+) %", lines[6]):
        line_6 = m.group(1).replace(',','')
    else: 
        line_6 = "NaN"

    # Line - 7
    if m := re.match(r"ROE ([0-9,.-]+) %", lines[7]):
        line_7 = m.group(1).replace(',','')
    else: 
        line_7 = "NaN"

    # Line - 8
    if m := re.match(r"Face Value ₹ ([0-9,.-]+)", lines[8]):
        line_8 = m.group(1).replace(',','')
    else: 
        line_8 = "NaN"

    x = [{
    "Market Cap in Cores Rupees":line_0,
    "Current Price in Rupees":line_1,
    "High in Rupees":line_2a,
    "Low in Rupees":line_2b,
    "Stock PE":line_3,
    "Book Value in Rupees":line_4,
    "Dividend Yield %": line_5,
    "ROCE %":line_6,
    "ROE %":line_7,
    "Face Value in Rupees":line_8,
    }]

    df = pd.DataFrame(x)

    df['Market Cap in Cores Rupees'] = df['Market Cap in Cores Rupees'].replace('','NaN').astype(float)
    df['Current Price in Rupees'] = df['Current Price in Rupees'].replace('','NaN').astype(float) #.astype("Int64")
    df['High in Rupees'] = df['High in Rupees'].replace('','NaN').astype(float) #.astype("Int64")
    df['Low in Rupees'] = df['Low in Rupees'].replace('','NaN').astype(float) #.astype("Int64")
    df['Stock PE'] = df['Stock PE'].replace('','NaN').astype(float) #.astype("Int64")
    df['Book Value in Rupees'] = df['Book Value in Rupees'].replace('','NaN').astype(float) #.astype("Int64")
    df['Dividend Yield %'] = df['Dividend Yield %'].replace('','NaN').astype(float) #.astype("Int64")
    df['ROCE %'] = df['ROCE %'].replace('','NaN').astype(float) #.astype("Int64")
    df['ROE %'] = df['ROE %'].replace('','NaN').astype(float) #.astype("Int64")
    df['Face Value in Rupees'] = df['Face Value in Rupees'].replace('','NaN').astype(float) #.astype("Int64")

    return df.copy()



# This function fisrt removes comma, percent and space characters 
# then fills all blanks with NaN 
# then converts all the column data to float 
def format_to_numeric(df,numeric_cols):
    for col in numeric_cols: 
        df[col] = df[col].str.replace(' ','').str.replace(',','').str.replace('%','').replace('','NaN').astype(float)

    return df.copy()



# Note: Banking and NBFC Stock Quarterly Result has 
# 'Revenue' instead of 'sales
# 'Financing Profit' instead of 'Operating Profit'
# 'Financing Margin %' instead of 'OPM %'
# This functions creates common terminology in order for them to be consistent
def fix_banks_nbfc_df_qr(temp_df):
    # Revenue, Sales --> Sales or Revenues
    temp_df['Quarterly Results'] = temp_df['Quarterly Results'].str.replace(r'(Sales|Revenue)', 'Sales or Revenue', regex=True)

    # Financing Profit, Operating Profit --> Finanacing or Operating Profit
    temp_df['Quarterly Results'] = temp_df['Quarterly Results'].str.replace(r'(Operating Profit|Financing Profit)', 'Financing or Operating Profit', regex=True)
    
    # OPM %, Financing Margin % --> 'Operating or Financing Profit Margin %'
    temp_df['Quarterly Results'] = temp_df['Quarterly Results'].str.replace(r'(OPM %|Financing Margin %)', 'Operating or Financing Profit Margin %', regex=True)
    return temp_df.copy()

# Note: Banking and NBFC Stock Yearly Result has 
# 'Revenue' instead of 'sales
# 'Financing Profit' instead of 'Operating Profit'
# 'Financing Margin %' instead of 'OPM %'
# This functions creates common terminology in order for them to be consistent
def fix_banks_nbfc_df_yr(temp_df):
    # Revenue, Sales --> Sales or Revenues
    temp_df['Profit and Loss'] = temp_df['Profit and Loss'].str.replace(r'(Sales|Revenue)', 'Sales or Revenue', regex=True)

    # Financing Profit, Operating Profit --> Finanacing or Operating Profit
    temp_df['Profit and Loss'] = temp_df['Profit and Loss'].str.replace(r'(Operating Profit|Financing Profit)', 'Financing or Operating Profit', regex=True)
    
     # OPM %, Financing Margin % --> 'Operating or Financing Profit Margin %'
    temp_df['Profit and Loss'] = temp_df['Profit and Loss'].str.replace(r'(OPM %|Financing Margin %)', 'Operating or Financing Profit Margin %', regex=True)
    
    return temp_df.copy()




In [51]:
writer = None
dir_path = "../Output/IndividualStocks/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"/"


if not os.path.exists(dir_path):
    os.makedirs(dir_path)

## Read entire page using Pandas

print("Loading Page for ",stock_symbol)
tables = pd.read_html(stock_screener_url)

time.sleep(1) # Seconds

# Read the Top Ration. Top Ratios section is a List Item, not a table hence need 
# to parse manually usiang bsoup 
response = rq.get(stock_screener_url)
# Parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")
# Find the unordered list
ul = soup.find("ul#top-ratios")  # You might need to use a more specific selector like ul.my-list
lines = []
for ul in soup.findAll('ul', id='top-ratios'):
    for li in ul.findAll('li'):
        li_parsed_text = li.text
        li_parsed_text = re.sub('[\s ]+', ' ', li_parsed_text)
        li_parsed_text = li_parsed_text.strip()
        #print(li_parsed_text)
        lines.append(li_parsed_text)
   
time.sleep(1) # Seconds


# Get all Tables in separate dataframes
df_top_ratios               = csvfy(lines) # Call function
df_quaterly_results         = tables[0] # Quarterly Results
df_profit_n_loss            = tables[1] # Profit & Loss
df_compounded_sales_growth  = tables[2] # Compounded Sales Growth
df_compounded_profit_growth = tables[3] # Compounded Profit Growth
df_stock_price_cagr         = tables[4] # Stock Price CAGR
df_return_on_equity         = tables[5] # Return on Equity
df_balance_sheet            = tables[6] # Balance Sheet
df_cash_flows               = tables[7] # Cash Flows
df_ratios                   = tables[8] # Ratios
df_shareholding_pattern     = tables[9] # Shareholding Pattern

# Cleanup table: Quarterly Results
df_quaterly_results.rename(columns={'Unnamed: 0':'Quarterly Results'}, inplace=True)
df_quaterly_results.replace(u"\u00A0\+", "", regex=True,inplace=True) 

# Cleanup table: Profit & Loss
df_profit_n_loss.rename(columns={'Unnamed: 0':'Profit and Loss'}, inplace=True)
df_profit_n_loss.replace(u"\u00A0\+", "", regex=True,inplace=True) 

# Cleanup table: Compounded Sales Growth
df_compounded_sales_growth.replace(":", "", regex=True,inplace=True) 

# Cleanup table: Compounded Profit Growth
df_compounded_profit_growth.replace(":", "", regex=True,inplace=True) 

# Cleanup table: Stock Price CAGR
df_stock_price_cagr.replace(":", "", regex=True,inplace=True) 

# Cleanup table: Return on Equity
df_return_on_equity.replace(":", "", regex=True,inplace=True) 

# Cleanup table: Balance Sheet
df_balance_sheet.rename(columns={'Unnamed: 0':'Balance Sheet'}, inplace=True)
df_balance_sheet.replace(u"\u00A0\+", "", regex=True,inplace=True) 

# Cleanup table: Cash Flows
df_cash_flows.rename(columns={'Unnamed: 0':'Cash Flows'}, inplace=True)
df_cash_flows.replace(u"\u00A0\+", "", regex=True,inplace=True) 

# Cleanup table: Ratios
df_ratios.rename(columns={'Unnamed: 0':'Ratios'}, inplace=True)

# Cleanup table: Shareholding Pattern
df_shareholding_pattern.rename(columns={'Unnamed: 0':'Shareholding Pattern'}, inplace=True)
df_shareholding_pattern.replace(u"\u00A0\+", "", regex=True,inplace=True)


sheet_names = ["Top Ratios","Quarterly Results", "Profit & Loss", "Compounded Sales Growth", "Compounded Profit Growth", 
               "Stock Price CAGR", "Return on Equity", "Balance Sheet", "Cash Flows", "Ratios", "Shareholding Pattern"]
dataframes  = [df_top_ratios, df_quaterly_results, df_profit_n_loss , df_compounded_sales_growth, df_compounded_profit_growth, 
               df_stock_price_cagr, df_return_on_equity, df_balance_sheet, df_cash_flows, df_ratios, df_shareholding_pattern]

writer = pd.ExcelWriter(dir_path + stock_symbol + ".xlsx" , engine='xlsxwriter')
for i, frame in enumerate(dataframes):
    frame.to_excel(writer, sheet_name = sheet_names[i], index=False)
writer.close()
writer.handles = None

print("Finished. Report Generated:", (dir_path + stock_symbol + ".xlsx") )

stock_report_file = "" + dir_path + stock_symbol + ".xlsx"

Loading Page for  ICICIBANK
Finished. Report Generated: ../Output/IndividualStocks/20240826-183514/ICICIBANK.xlsx


In [53]:
df_top_ratios               = csvfy(lines) # Call function
df_quaterly_results         = tables[0] # Quarterly Results
df_profit_n_loss            = tables[1] # Profit & Loss
df_compounded_sales_growth  = tables[2] # Compounded Sales Growth
df_compounded_profit_growth = tables[3] # Compounded Profit Growth
df_stock_price_cagr         = tables[4] # Stock Price CAGR
df_return_on_equity         = tables[5] # Return on Equity
df_balance_sheet            = tables[6] # Balance Sheet
df_cash_flows               = tables[7] # Cash Flows
df_ratios                   = tables[8] # Ratios
df_shareholding_pattern     = tables[9] # Shareholding Pattern


# Convert to numeric: 
df_quaterly_results = format_to_numeric(df_quaterly_results.copy(),df_quaterly_results.columns[1:])
df_quaterly_results = fix_banks_nbfc_df_qr(df_quaterly_results)

# Convert to numeric: 
df_profit_n_loss = format_to_numeric(df_profit_n_loss.copy(),df_profit_n_loss.columns[1:])
df_profit_n_loss = fix_banks_nbfc_df_yr(df_profit_n_loss)

In [54]:
# df_top_ratios # all good 

In [58]:
df_compounded_sales_growth

,Compounded Sales Growth,Compounded Sales Growth.1
0,10 Years,12%
1,5 Years,18%
2,3 Years,22%
3,TTM,25%


In [59]:
df_compounded_sales_growth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Compounded Sales Growth    4 non-null      object
 1   Compounded Sales Growth.1  4 non-null      object
dtypes: object(2)
memory usage: 196.0+ bytes


In [26]:
df1 = format_to_numeric(df_profit_n_loss.copy(),df_profit_n_loss.columns[1:])
df1

,Profit and Loss,Mar 2013,Mar 2014,Mar 2015,Mar 2016,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Mar 2024,TTM
0,Sales,11833.0,13415.00,15023.00,15568.00,14196.00,14281.00,15448.00,17517.00,19048.00,21545.00,24670.00,28011.00,28949.00
1,Expenses,9164.0,10164.00,11515.00,11983.00,11724.00,11930.00,12270.00,15040.00,15173.00,17778.00,18200.00,20078.00,20957.00
2,Operating Profit,2669.0,3251.00,3508.00,3585.00,2472.00,2351.00,3178.00,2477.00,3874.00,3768.00,6470.00,7933.00,7992.00
3,OPM %,23.0,24.00,23.00,23.00,17.00,16.00,21.00,14.00,20.00,17.00,26.00,28.00,28.00
4,Other Income,146.0,170.00,260.00,295.00,172.00,155.00,381.00,670.00,335.00,555.00,971.00,909.00,932.00
5,Interest,100.0,127.00,108.00,83.00,63.00,79.00,89.00,98.00,97.00,96.00,143.00,171.00,194.00
6,Depreciation,550.0,648.00,760.00,939.00,1027.00,1077.00,1135.00,1163.00,1229.00,1165.00,1250.00,1470.00,1497.00
7,Profit before tax,2165.0,2646.00,2900.00,2859.00,1554.00,1350.00,2336.00,1886.00,2884.00,3061.00,6048.00,7201.00,7234.00
8,Tax %,29.0,26.00,19.00,26.00,19.00,32.00,17.00,-7.00,32.00,29.00,25.00,23.00,NaN
9,Net Profit,1527.0,1963.00,2336.00,2131.00,1292.00,947.00,1950.00,2026.00,1952.00,2182.00,4507.00,5578.00,5565.00


In [25]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Profit and Loss  12 non-null     object 
 1   Mar 2013         12 non-null     float64
 2   Mar 2014         12 non-null     float64
 3   Mar 2015         12 non-null     float64
 4   Mar 2016         12 non-null     float64
 5   Mar 2017         12 non-null     float64
 6   Mar 2018         12 non-null     float64
 7   Mar 2019         12 non-null     float64
 8   Mar 2020         12 non-null     float64
 9   Mar 2021         12 non-null     float64
 10  Mar 2022         12 non-null     float64
 11  Mar 2023         12 non-null     float64
 12  Mar 2024         12 non-null     float64
 13  TTM              10 non-null     float64
dtypes: float64(13), object(1)
memory usage: 1.4+ KB
